In [1]:
import time
import lxml
import bs4
import requests
BeautifulSoup = bs4.BeautifulSoup
import time
import random
import re
#上面是基本上没什么卵用了已经的静态页面爬虫
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
#这里是自动化操作浏览器的工具
import cv2
import pytesseract
#这里是处理截图的工具
import pandas as pd
import csv

我负责的部分是湘菜和徽菜。我主要是想使用selenium来进行。我不打算使用记录每一个页面的规律，而是采用模拟点击翻页的方法来进行。也是要挨个儿点进每个菜谱的具体页面爬取。关于截图然后opencv，这个看情况看用不用的上。毕竟opencv的截图方法姑且还是基于概率的，即使正确率再高，比起直接从页面拿到数据，还是有错误的可能的

In [2]:
#姑且还是把请求头拿过来。
user_agents = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]
#随机拿一个
request_headers = {
    'user-agent':
        random.choice(user_agents),
    'Connection':
        "keep-alive",
    'Referer':
        "https://www.douban.com"
}

In [3]:
receipt_list_url = {
    '湘菜':'https://home.meishichina.com/recipe/xiangcai/',
    '徽菜':'https://home.meishichina.com/recipe/huicai/'
}

为了不让数据量beyond what we can manage,我们打算每个菜系爬30道，也就是三页。

模拟人类操作，统一在所有点击操作之前进行sleep。

In [5]:
driver = webdriver.Edge()

现在先开始分析xpath。

第一个菜的连接的xpath是：//*[@id="J_list"]/ul/li[1]/div[2]/h2/a

第一个菜的连接的xpath是：//*[@id="J_list"]/ul/li[2]/div[2]/h2/a

显然由于一页有10个详情页，所以['//*[@id="J_list"]/ul/li[%d]/div[2]/h2/a'%i for i in range(1,11)]

In [6]:
def wait(second):
    time.sleep(random.uniform(0.7,0.7 + second))

In [7]:
def img_2_text(driver,target_element,img_path):
    # 使用 ActionChains 将鼠标悬停在目标元素上，以确保其完全可见
    ActionChains(driver).move_to_element(target_element).perform()
    target_element.screenshot(img_path)

    
    image = cv2.imread(img_path)
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(image_rgb, lang="chi_sim")

#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)


#     recognized_text = pytesseract.image_to_string(gray_image, lang='chi_sim')
    return text

In [8]:
def get_specific_receipt_info(driver, now_url):
    resp = requests.get(url = now_url,headers = request_headers)
    status_code = resp.status_code
    if status_code == 200:
        resp.encoding = 'utf-8'
        html_text = resp.text
        soup_document = BeautifulSoup(html_text, 'lxml')
        selector = lxml.etree.HTML(html_text)
        
        
        #-----------------------------即使没有必要，我这里也打算用多种不同的方式来拿到数据---------------------
        
        #菜品名字使用css选择器来找
        #这是菜品名字#recipe_title
        dish_name = soup_document.select('#recipe_title')[0].get_text().strip()
        

        
        #然后是主料辅料和调料
        #首先是主料
        #ingredients_loader = soup_document.select('body > div.wrap > div > div.space_left > div.space_box_home > div > fieldset[1] > div > ul > li > span.category_s1 > a > b')
        #主料必须用xpath而不是css。因为不同页面主料对应的fieldset是不一样的。有的是7.有的是8.但是xpath这里都是从1到3，有规律可循
        main_ingredients_loader = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li/span[1]/a/b')
        main_ingredient_dict = {}
        for i in range(0,len(main_ingredients_loader)):
            wait(1)
            main_ingredient_name = main_ingredients_loader[i].text
            
            picture_xpath = '/html/body/div[5]/div/div[1]/div[3]/div/fieldset[1]/div/ul/li[%d]/span[2]'%(i+1)
            #好吧，看来还是用上了opencv。这里不知道为什么用量截取不下来，那么我只好截图了（喜）
            #首先给具体数量截个图
            #啊啊啊啊啊气死我了适量识别不出来,好在适量可以手动添加
            amount_picture_element = driver.find_element(By.XPATH, picture_xpath)
            amount_picture_text = img_2_text(driver,amount_picture_element,'main_ingredient_amount.png').replace('\n','')
            main_ingredient_dict[main_ingredient_name] = amount_picture_text if amount_picture_text else '适量'
        #print(main_ingredient_dict)
                                                    
        secondary_ingredients_loader = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li/span[1]/a/b')
        secondary_ingredient_dict = {}
        for i in range(0,len(secondary_ingredients_loader)):
            wait(1)
            secondary_ingredient_name = secondary_ingredients_loader[i].text
            
            picture_xpath = '/html/body/div[5]/div/div[1]/div[3]/div/fieldset[2]/div/ul/li[%d]/span[2]'%(i+1)
            amount_picture_element = driver.find_element(By.XPATH, picture_xpath)
            amount_picture_text = img_2_text(driver,amount_picture_element,'secondary_ingredient_amount.png').replace('\n','')
            secondary_ingredient_dict[secondary_ingredient_name] = amount_picture_text if amount_picture_text else '适量'
        #print(secondary_ingredient_dict)
        
        
        spices_loader = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li/span[1]/a/b')
        spices_dict = {}
        for i in range(0,len(spices_loader)):
            wait(1)
            spices_name = spices_loader[i].text
            
            picture_xpath = '/html/body/div[5]/div/div[1]/div[3]/div/fieldset[3]/div/ul/li[%d]/span[2]'%(i+1)
            amount_picture_element = driver.find_element(By.XPATH, picture_xpath)
            amount_picture_text = img_2_text(driver,amount_picture_element,'spices__amount.png').replace('\n','')
            spices_dict[spices_name] = amount_picture_text if amount_picture_text else '适量'
        #print(spices_dict)
        
        
        #接下来是特点。这个东西好就好在非常固定，一共只有：
        #口味 工艺 耗时 难度
        detail_info = {
            '口味':None,
            '工艺':None,
            '耗时':None,
            '难度':None
        }
        #这是口味/html/body/div[5]/div/div[1]/div[3]/div/div[4]/ul/li[1]/span[1]/a
        #这是工艺/html/body/div[5]/div/div[1]/div[3]/div/div[4]/ul/li[2]/span[1]/a
        #可以看到还是有规律的
        detail_info_index = 1
        for i in detail_info:
            wait(1)
            detail_text = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[4]/ul/li[%d]/span[1]/a'%detail_info_index)[0].text
            detail_info[i] = detail_text
            detail_info_index += 1
        
        
        
        wait(1)
        #最后是描述文本了。
        #这个就是我们想要的东西。/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/text()
        cooking_tutorial_text = ''
        now_step = 1
        cooking_tutorial_steps = selector.xpath('/html/body/div[5]/div/div[1]/div[3]/div/div[6]/ul/li/div[2]/text()')
        for cooking_tutorial_step in cooking_tutorial_steps:
            now_step_string = '%d、'%now_step
            cooking_tutorial_text += now_step_string
            cooking_tutorial_text += cooking_tutorial_step
            cooking_tutorial_text += '  '
            now_step += 1
    
        #好的现在我们已经拿到了所有需要的东西了。
        data_return = (dish_name,main_ingredient_dict,secondary_ingredient_dict,spices_dict,detail_info,cooking_tutorial_text)
        return data_return
        
        
    else:
        print('fuck')
        return None
        
    

In [15]:
def write_csv(data_list, cuisine_type, a,b,c):
    #先研究一下按什么格式写入csv。

        
    #需要定义一个ingredients_have_been_added,防止重复装入相同的原料
    main_ingredient_have_been_added = a
    secondary_ingredient_have_been_added = b
    spices_dict_have_been_added = c

    for i in data_list:
        with open('dish.csv', 'a', newline='',encoding='utf-8') as f: 
            writer = csv.writer(f)
            writer.writerow([i[0],i[4]['口味'],i[4]['工艺'],i[4]['耗时'],i[4]['难度'],i[5]])
            
            
        with open('main_ingredient.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for main_ingredient_name in i[1]:
                if main_ingredient_name in main_ingredient_have_been_added:
                    continue
                main_ingredient_have_been_added.append(main_ingredient_name)
                writer.writerow([main_ingredient_name])

    
        with open('secondary_ingredient.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for secondary_ingredient_name in i[2]:
                if secondary_ingredient_name in secondary_ingredient_have_been_added:
                    continue
                secondary_ingredient_have_been_added.append(secondary_ingredient_name)
                writer.writerow([secondary_ingredient_name])
            
                
        with open('spices.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for spices_name in i[3]:
                if spices_name in spices_dict_have_been_added:
                    continue
                spices_dict_have_been_added.append(spices_name)
                writer.writerow([spices_name])
                

        with open('ingredient_amount.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            for main_ingredient_name in i[1]:
                writer.writerow([main_ingredient_name,i[1][main_ingredient_name],i[0]])
            for secondary_ingredient_name in i[2]:
                writer.writerow([secondary_ingredient_name,i[2][secondary_ingredient_name],i[0]])
            for spices_name in i[3]:
                writer.writerow([spices_name,i[3][spices_name],i[0]])

        with open('cuisine_dish.csv','a',newline='',encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([i[0],'属于',cuisine_type])
            
    return (main_ingredient_have_been_added,secondary_ingredient_have_been_added,spices_dict_have_been_added)

这里比较生草的地方是，这个热门按钮按了之后就不再可交互了。我之前在这个地方老是告诉我找不到元素。之前爬哔哩哔哩真的给孩子爬出心理阴影了，现在看见这个找不到元素的报错就真的想哭。

In [10]:
def get_receipt(receipt_url):
    all_data = []
    driver.get(receipt_url)
    #先点击热门。我打算找最热门的30个。
    #热门按钮的xpath是：/html/body/div[5]/div/div[1]/div[1]/div/a[1]
    try:
        wait(1)
        driver.find_element (By.XPATH, '/html/body/div[5]/div/div[1]/div[1]/div/a[1]').click()
    except Exception:
        print('已经是最热了')
    for _ in range(3):
        #翻页什么的在函数末尾在考虑
        
        #首先在这里拿到我们当前的列表页的记录。
        current_window_handle = driver.current_window_handle
        
        #现在要把这10个详情页挨个儿点进去。
        content_pages = ['//*[@id="J_list"]/ul/li[%d]/div[2]/h2/a'%i for i in range(1,11)]
        picture_index = 1
        for content_page in content_pages:
            wait(1)
            driver.find_element (By.XPATH, content_page).click()
            #把driver切换到新的页面
            all_window_handles = driver.window_handles
            for window_handle in all_window_handles:
                if window_handle != current_window_handle:
                    wait(2)
                    driver.switch_to.window(window_handle)
                    break
            
            
            
            #上面的部分是实现翻页点击页面相关
            #--------------------------------------------------------------------------------------
            #这里是对页面进行具体爬取。我比较喜欢先验证了这个自动翻页进入页面的可行性。因为之前
            #爬蜀黍的b站真的给孩子爬ptsd了。所以真的讨厌太正规的网站、、那么羞涩干什么怎么就不让我
            #好好爬呢（恼），，，作为用户的时候责怪b站怎么不把自己搞得安全一点三天两头被黑，需要爬
            #它的时候就想着这玩意怎么这么固若金汤、、这就是人性吗（悲）
            now_url = driver.current_url
            this_data = get_specific_receipt_info(driver, now_url)
            all_data.append(this_data)
            
            
            
            
            #--------------------------------------------------------------------------------------
            #下面的部分是实现翻页点击页面相关
            
            #结束之后把当前页面关掉
            wait(1)
            driver.close()
            #切换回列表页
            wait(1)
            driver.switch_to.window(current_window_handle)
            
            
            dish_name = this_data[0]
            #菜品图片在列表页扒。因为详情页的不太合适
            #切换回来之后就可以截图了。因为现在拿到了菜品的名字
            #这是图片的xpath： //*[@id="J_list"]/ul/li[1]/div[1]/a/img
            dish_picture_xpath = '//*[@id="J_list"]/ul/li[%d]/div[1]/a/img'%picture_index
            picture_index += 1
            dish_picture_element = driver.find_element(By.XPATH, dish_picture_xpath)
            # 使用 ActionChains 将鼠标悬停在目标元素上，以确保其完全可见
            ActionChains(driver).move_to_element(dish_picture_element).perform()
            dish_picture_element.screenshot('%s.jpg'%dish_name)
            
            #break
        
        #这里是翻页
        #翻页的xpath：/html/body/div[5]/div/div[1]/div[3]/div/a[6]
        wait(1)
        driver.find_element (By.XPATH, '/html/body/div[5]/div/div[1]/div[3]/div/a[6]').click()
        #break
        
    return all_data

In [11]:
caipu = []
for i in receipt_list_url:
    this_url = receipt_list_url[i]
    caipu.append(get_receipt(this_url))


C:\Users\ft183\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py:343: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn(


In [48]:
caipu[0]

[('【湘菜】啤酒鸭——夏季里降火的大肉菜',
  {'鸭': '半只', '啤酒': '600ml'},
  {'青红椒': '适量', '葱': '适量', '姜': '适量', '蒜': '适量'},
  {'生抽': '适量', '老抽': '适量', '糖': '适量', '盐': '适量', '八角': '适量'},
  {'口味': '微辣', '工艺': '炖', '耗时': '一小时', '难度': '普通'},
  '1、准备材料。  2、鸭洗净切块，辣椒切块。  3、炒锅热油，放干辣椒、姜、蒜爆香，放鸭肉翻炒。  4、大火煸炒至鸭肉收干不出水（出水多可以倒掉），并且炒出部分鸭油。  5、加生抽、老抽、糖、一个八角炒至上色。  6、加入一瓶啤酒。  7、大火烧开后，改中小火焖三十分钟。  8、待汤汁基本收干加青红椒、葱花翻炒，再加盐调味，即可起锅。  9、趁热吃吧。  ')]

In [16]:
main_ingredient_have_been_added = []
secondary_ingredient_have_been_added = []
spices_dict_have_been_added = []

    
#首先是要把所有的菜品名写入。菜品名需要的下表是：0，4，5
with open('dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','口味','工艺','耗时','难度','步骤'])

with open('main_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open('secondary_ingredient.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open('spices.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['名字'])

with open('ingredient_amount.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['原料名','用量','菜品名'])

with open('cuisine_dish.csv','w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['菜品名','属于','菜系'])
        
for i,cusion_type in zip(caipu,receipt_list_url):
    (main_ingredient_have_been_added,secondary_ingredient_have_been_added,spices_dict_have_been_added) = write_csv(i,cusion_type,main_ingredient_have_been_added,secondary_ingredient_have_been_added,spices_dict_have_been_added)

在这个地方，我爬了1个小时。我不敢在这个ipynb的基础上改了。这个版本爬下来的图片有点问题，我将在另一个ipynb文件中爬取图片。